In [31]:
import numpy as np
from sklearn.linear_model import Perceptron
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder , StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score,accuracy_score
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from collections import Counter


In [ ]:
diabetes = pd.read_csv("diabetes_012_health_indicators_BRFSS2015.csv")
# print(diabetes['Diabetes_012'].value_counts())
# diabetes['Diabetes_012'] = diabetes['Diabetes_012'].replace(2, 1)
diabetes.head()
# print(diabetes['Diabetes_012'].value_counts())


Diabetes_012
0.0    213703
2.0     35346
1.0      4631
Name: count, dtype: int64
Diabetes_012
0.0    213703
1.0     39977
Name: count, dtype: int64


In [41]:

x_train, x_test = train_test_split(diabetes, test_size=0.2, random_state=42)
x_test,y_test = x_test.drop(['Diabetes_012'],axis=1),x_test['Diabetes_012']

# x_train, x_val = train_test_split(x_train, test_size=0.20, random_state=42)

x_train,y_train = x_train.drop(['Diabetes_012'],axis=1),x_train['Diabetes_012']
# x_val,y_val = x_val.drop(['Diabetes_012'],axis=1),x_val['Diabetes_012']


In [ ]:
from sklearn.svm import SVC


svm = Pipeline([
    ('scaler', StandardScaler()),
    ('model', SVC(probability=True,kernel='linear', class_weight='balanced'
                  ,random_state=42
                    ,C=0.1,gamma='scale',degree=3)
                  )
])



# grid_search = GridSearchCV(svm, params, cv=5, scoring='accuracy')
# 4. Run grid search
svm.fit(x_train, y_train)

# 5. Show results
# print("Best parameters:", grid_search.best_params_)
# print("Best cross-validation score: {:.2%}".format(grid_search.best_score_))

# 6. Evaluate on test set
# best_model = grid_search.best_estimator_
y_pred = svm.predict(x_test)
print("\nTest Accuracy: {:.2%}".format(accuracy_score(y_test, y_pred)))

In [42]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('perceptron', Perceptron(
        random_state=42,
        max_iter=1000,
        alpha=0.1,
        penalty='l1',
        # 'eta0': 0.1,
        early_stopping=True,
        ))
])

# Fit the pipeline to the training data
pipe.fit(x_train, y_train)
# Make predictions on the test set
y_pred = pipe.predict(x_test)
# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(y_test.value_counts())
print(y_train.value_counts())
print( Counter(y_pred))
# print( Counter(y_pred_test))

Accuracy: 0.8434839167455062
Diabetes_012
0.0    42795
1.0     7941
Name: count, dtype: int64
Diabetes_012
0.0    170908
1.0     32036
Name: count, dtype: int64
Counter({np.float64(0.0): 50736})


In [ ]:
# 1. FIRST ensure you have the right imports
from imblearn.pipeline import Pipeline  # CRITICAL - must use imblearn's Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from imblearn.over_sampling import SMOTE

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),  # Will only be applied to training folds
    ('perceptron', Perceptron(random_state=42))
])

# 4. Define a simplified parameter grid (to ensure it runs first)
param_grid = {
    'smote__sampling_strategy': [{1: x} for x in [len(y_train[y_train==0])//3]],  # Start with one value
    'perceptron__class_weight': ['balanced'],  # Start simple
    'perceptron__penalty': ['l2'],  # Start with basic penalty
    'perceptron__alpha': [0.001, 0.01],  # Reduced options
    'perceptron__max_iter': [1000]
}

# 5. Create GridSearchCV
grid_search = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,  # Reduced folds for initial test
    n_jobs=1,  # Start with single job
    verbose=3
)

# 6. Run grid search
print("Starting grid search...")
grid_search.fit(x_train, y_train)

# 7. Show results
print("\nBest parameters:", grid_search.best_params_)
print("Best CV accuracy: {:.2%}".format(grid_search.best_score_))

# 8. Final evaluation
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test)
print("\nTest Accuracy: {:.2%}".format(accuracy_score(y_test, y_pred)))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Class distribution before SMOTE: {np.float64(0.0): np.int64(170908), np.float64(1.0): np.int64(3687), np.float64(2.0): np.int64(28349)}
Starting grid search...
Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV 1/3] END perceptron__alpha=0.001, perceptron__class_weight=balanced, perceptron__max_iter=1000, perceptron__penalty=l2, smote__sampling_strategy={1: 56969};, score=0.695 total time=   1.1s
[CV 2/3] END perceptron__alpha=0.001, perceptron__class_weight=balanced, perceptron__max_iter=1000, perceptron__penalty=l2, smote__sampling_strategy={1: 56969};, score=0.577 total time=   1.1s
[CV 3/3] END perceptron__alpha=0.001, perceptron__class_weight=balanced, perceptron__max_iter=1000, perceptron__penalty=l2, smote__sampling_strategy={1: 56969};, score=0.563 total time=   1.5s
[CV 1/3] END perceptron__alpha=0.01, perceptron__class_weight=balanced, perceptron__max_iter=1000, perceptron__penalty=l2, smote__sampling_strategy={1: 56969};, score=0.507 total time=   1.0s
[CV 2/3] E

In [ ]:

preprocessor = StandardScaler()
X_train_scaled = preprocessor.fit_transform(x_train)
X_test_scaled = preprocessor.transform(x_test)

# Apply SMOTE only to training data
smote = SMOTE(sampling_strategy={1: len(y_train[y_train==0])//3})
X_res, y_res = smote.fit_resample(X_train_scaled, y_train)

# Train Perceptron
model = Perceptron(
    class_weight={0:1, 1:10, 2:3},
    penalty='l1',
    alpha=0.1,
    max_iter=1000,
    random_state=42
)
model.fit(X_res, y_res)

# Evaluate
y_pred = model.predict(X_test_scaled)
print("\nTest Accuracy: {:.2%}".format(accuracy_score(y_test, y_pred)))

print(y_test.value_counts())
print(y_train.value_counts())
print( Counter(y_pred_train))
print( Counter(y_pred_test))


Test Accuracy: 37.89%


In [ ]:


# For predictions (NumPy arrays)
print("\nPredicted class distribution:")
print("Training predictions:", Counter(y_pred))



Predicted class distribution:
Training predictions: Counter({np.float64(1.0): 21611, np.float64(0.0): 17714, np.float64(2.0): 11411})


In [26]:
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import Perceptron
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from imblearn.pipeline import Pipeline
import numpy as np

# First, analyze your class distribution
print("Original class distribution:", np.bincount(y_train))

# Create a safer SMOTE configuration
def get_smote_strategy(y):
    counts = np.bincount(y)
    minority_class = np.argmin(counts)
    return {minority_class: min(counts[minority_class]*2, max(counts))}  # At most double minority samples

# Create pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('model', Perceptron(random_state=42))
])

# Safer parameter grid
param_grid = {
    'model__class_weight': ['balanced', {0:1, 1:3, 2:2}],  # Simplified weights
    'model__penalty': ['l1', 'l2'],
    'model__alpha': [0.001, 0.01, 0.1],
    'model__max_iter': [1000, 2000],
    'smote__sampling_strategy': [
        'auto',
        'minority',
        get_smote_strategy(y_train)
    ]
}

# Reduced GridSearchCV configuration
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,  # Fewer folds for stability
    n_jobs=-1,
    verbose=2,
    error_score='raise'
)

try:
    grid_search.fit(x_train, y_train)
    
    # Output results
    print("Best Parameters:", grid_search.best_params_)
    y_pred = grid_search.best_estimator_.predict(x_test)
    print("\nTest Accuracy: {:.2%}".format(accuracy_score(y_test, y_pred)))
    
except ValueError as e:
    print(f"Error encountered: {e}")
    print("Try adjusting the SMOTE sampling strategy or class weights")

C:\Users\LENOVE\AppData\Local\Temp\ipykernel_25808\952081348.py:10: DeprecationWarning: Non-integer input passed to bincount. In a future version of NumPy, this will be an error. (Deprecated NumPy 2.1)
  print("Original class distribution:", np.bincount(y_train))
C:\Users\LENOVE\AppData\Local\Temp\ipykernel_25808\952081348.py:14: DeprecationWarning: Non-integer input passed to bincount. In a future version of NumPy, this will be an error. (Deprecated NumPy 2.1)
  counts = np.bincount(y)


Original class distribution: [170908   3687  28349]
Fitting 3 folds for each of 72 candidates, totalling 216 fits
Best Parameters: {'model__alpha': 0.1, 'model__class_weight': {0: 1, 1: 3, 2: 2}, 'model__max_iter': 1000, 'model__penalty': 'l1', 'smote__sampling_strategy': {np.int64(1): np.int64(7374)}}

Test Accuracy: 84.35%


In [ ]:
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import Perceptron
from imblearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import numpy as np


perceptron_params = {
    'alpha': 0.01,  # Reduced regularization
    'class_weight': {0: 3, 1: 7, 2: 3},  # More aggressive weighting
    'max_iter': 10000,  # More iterations
    'penalty': 'l1',  # Try L2 instead of L1
    'random_state': 42,
    'eta0': 0.1,  # Higher learning rate
    'early_stopping': True,
    'validation_fraction': 0.1
}

# More balanced SMOTE configuration
smote_params = {
    'sampling_strategy': {1: 15000, 2: 30000},  # More balanced approach
    'random_state': 42,
    'k_neighbors': 5
}

# Create pipeline with verbose output
optimized_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(**smote_params)),
    ('classifier', Perceptron(**perceptron_params))
], verbose=True)

# Fit with validation
print("\nFitting model...")
optimized_pipeline.fit(x_train, y_train)

# Check resampled class distribution
if hasattr(optimized_pipeline.named_steps['smote'], 'sample_indices_'):
    print("\nResampled class distribution:", 
          np.bincount(optimized_pipeline.named_steps['smote'].sample_indices_))

# Predictions
y_pred_train = optimized_pipeline.predict(x_train)
y_pred_test = optimized_pipeline.predict(x_test)

# Evaluation
print("\nTraining Results:")
print("Accuracy:", accuracy_score(y_train, y_pred_train))
# print(classification_report(y_train, y_pred_train))

print("\nTest Results:")
print("Accuracy:", accuracy_score(y_test, y_pred_test))
# print(classification_report(y_test, y_pred_test))

# Check decision function
if hasattr(optimized_pipeline.named_steps['classifier'], 'decision_function'):
    print("\nDecision function samples:", 
          optimized_pipeline.named_steps['classifier'].decision_function(x_test[:5]))

C:\Users\LENOVE\AppData\Local\Temp\ipykernel_25792\2811015241.py:9: DeprecationWarning: Non-integer input passed to bincount. In a future version of NumPy, this will be an error. (Deprecated NumPy 2.1)
  print("Original class distribution:", np.bincount(y_train))


Original class distribution: [170908   3687  28349]

Fitting model...
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.1s
[Pipeline] ............. (step 2 of 3) Processing smote, total=   1.1s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   1.7s

Training Results:
Accuracy: 0.7643537133396405

Test Results:
Accuracy: 0.7638757489750867

Decision function samples: [[0.2        0.18683718 0.66692839]
 [0.05340987 0.26582571 0.81404856]
 [0.2        0.18683718 0.22230946]
 [0.2        0.18683718 0.10287752]
 [0.2        0.18683718 1.00100463]]


c:\Users\LENOVE\anaconda3\envs\myenv\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but Perceptron was fitted without feature names
  warnings.warn(


In [42]:
print(y_test.value_counts())
print(y_train.value_counts())
print( Counter(y_pred_train))
print( Counter(y_pred_test))

Diabetes_012
0.0    42795
2.0     6997
1.0      944
Name: count, dtype: int64
Diabetes_012
0.0    170908
2.0     28349
1.0      3687
Name: count, dtype: int64
Counter({np.float64(1.0): 101966, np.float64(0.0): 99874, np.float64(2.0): 1104})
Counter({np.float64(1.0): 25731, np.float64(0.0): 24730, np.float64(2.0): 275})
